In [25]:
#Necesary imports.
import bitso
import pandas as pd
import numpy as np
from datetime import timezone
import datetime
from dateutil.tz import tzutc
import time
from fbprophet import Prophet
import matplotlib.pyplot as plt
from scipy.stats import boxcox
import warnings
warnings.filterwarnings('ignore')
import plotly.offline
from plotly import graph_objs as go

In [26]:
#Opening my api key and secret.
with open('../key.txt') as f:
    a = f.readlines()

with open('../secret.txt') as p:
    b = p.readlines()

#Using the api secret and key to call the api.
API_SECRET = [i.split('\n') for i in a][0][0]
API_KEY = [i.split('\n') for i in b][0][0]
api = bitso.Api(API_KEY, API_SECRET)

In [27]:
'''
marker for last_scrap on 1-1-20: 13059700, I save this marker so i can use it
the next time i'm scrapping
'''
#Defining the empyt lists of the elements we need from the api.
btc_mxn_date = []
btc_mxn_price = []
btc_mxn_volume = []
btc_mxn_hour = []
btc_mxn_side = []

#Marker than changes the time in the api calls.
mark = 13059700

#Datetime when the code is running
now = datetime.datetime.now(timezone.utc)

#While loop for scrapping the information we need
while 1:
    #limit of requests, the max we can get is 100
    limit = '100'
    #Calling the api trades, we use sort 'asc' so i't starts on the marker we want.
    trades = api.trades('btc_mxn', sort='asc', limit=limit, marker=mark)
    #Starting the counter so we don't raise the error of too many requests on the api.
    count = 0
    #printing the mark and the spaces, we need these so we know the last mark we use.
    print(mark)
    print('')
    print('')
    #if statement to break the while loop.
    if trades != []:
        #starting a for loop to append the elements we need from each trade to our lists
        for e in trades:
            #adding to the counter.
            count +=1
            # if statement so we just scrap elements before today.
            if e.created_at < now :
                #printing each trade so we can see that everythings is working fine.
                print(e)
                #scrapping the elements we need
                btc_mxn_price.append(e.price)
                btc_mxn_volume.append(e.amount)
                btc_mxn_hour.append(e.created_at.hour)
                btc_mxn_date.append(e.created_at.date())
                btc_mxn_side.append(e.maker_side)
        '''
        if statement that starts a timer when the count reachs 100, we can just get
        300 requests per minute so our timer waits 21 seconds before making the next
        100 requests. It also adds a thousand to our mark so it moves forward in time.
        '''
        if count == 100:
            time.sleep(21)
            count = 0
            mark = mark + 1000
            
    #else statement to break the loop.
    else:
        break
        

13059700


Trade(tid=13059703, price=137185.86, amount=0.00072894, maker_side=sell, created_at=2020-01-01 23:54:46+00:00)
Trade(tid=13059704, price=137185.71, amount=0.00036447, maker_side=sell, created_at=2020-01-01 23:54:50+00:00)
Trade(tid=13059706, price=136500.00, amount=0.03215568, maker_side=buy, created_at=2020-01-01 23:55:03+00:00)
Trade(tid=13059707, price=137185.12, amount=0.00036447, maker_side=sell, created_at=2020-01-01 23:55:04+00:00)
Trade(tid=13059714, price=137184.27, amount=0.04373679, maker_side=sell, created_at=2020-01-01 23:55:28+00:00)
Trade(tid=13059720, price=136500.00, amount=0.00737394, maker_side=buy, created_at=2020-01-01 23:56:04+00:00)
Trade(tid=13059721, price=137182.25, amount=0.00075039, maker_side=sell, created_at=2020-01-01 23:56:20+00:00)
Trade(tid=13059722, price=137181.33, amount=0.02186887, maker_side=sell, created_at=2020-01-01 23:56:46+00:00)
Trade(tid=13059727, price=137180.52, amount=0.02831306, maker_side=sell, created_at=2020-01-01 23:57:09

13060700


Trade(tid=13060701, price=134667.64, amount=0.00063427, maker_side=buy, created_at=2020-01-02 03:56:38+00:00)
Trade(tid=13060702, price=134667.65, amount=0.00063412, maker_side=sell, created_at=2020-01-02 03:56:38+00:00)
Trade(tid=13060703, price=134667.65, amount=0.00043065, maker_side=buy, created_at=2020-01-02 03:56:38+00:00)
Trade(tid=13060704, price=134667.64, amount=0.00031713, maker_side=buy, created_at=2020-01-02 03:56:41+00:00)
Trade(tid=13060705, price=134667.67, amount=0.00021532, maker_side=buy, created_at=2020-01-02 03:56:44+00:00)
Trade(tid=13060706, price=135266.83, amount=0.00147856, maker_side=sell, created_at=2020-01-02 03:56:48+00:00)
Trade(tid=13060707, price=134667.64, amount=0.00055318, maker_side=buy, created_at=2020-01-02 03:57:24+00:00)
Trade(tid=13060708, price=134667.64, amount=0.00027659, maker_side=buy, created_at=2020-01-02 03:57:26+00:00)
Trade(tid=13060709, price=134667.65, amount=0.00028487, maker_side=buy, created_at=2020-01-02 03:57:29+00:0

13061700




In [32]:
# creating a new data frame.
df = pd.DataFrame(columns=['price', 'volume', 'hour', 'date', 'side'])
# using the lists we got from the while loop to fill the dataframe.
df.price = btc_mxn_price
df.volume = btc_mxn_volume
df.hour = btc_mxn_hour
df.date = btc_mxn_date
df.side = btc_mxn_side
df.date = df.date.astype('datetime64')
#creating new columns with the month and day of each row.
df['month'] = [i.month for i in df.date]
df['day'] = [i.day for i in df.date]
df.head(10)

,price,volume,hour,date,side,month,day
0,137185.86,0.00072894,23,2020-01-01,sell,1,1
1,137185.71,0.00036447,23,2020-01-01,sell,1,1
2,136500.00,0.03215568,23,2020-01-01,buy,1,1
3,137185.12,0.00036447,23,2020-01-01,sell,1,1
4,137184.27,0.04373679,23,2020-01-01,sell,1,1
5,136500.00,0.00737394,23,2020-01-01,buy,1,1
6,137182.25,0.00075039,23,2020-01-01,sell,1,1
7,137181.33,0.02186887,23,2020-01-01,sell,1,1
8,137180.52,0.02831306,23,2020-01-01,sell,1,1
9,137180.45,0.00072897,23,2020-01-01,sell,1,1


In [33]:
#Creating a function that needs a data frame and a number(month) for input.
def extraction(df, m):
    #variable that get's just the row that matches the same month
    f_df = df.loc[df['month'] == m]
    #variable that stores the year
    y = datetime.datetime.today().year
    #variable that creates a list with the numbers of days of the month
    days = list(f_df['day'].unique())
    #varible that creates a empty dictionary
    results = {}
    #list with the names of the keys in the dictionary that the function returns.
    nombres = ['date', '24h_avg', '12h_avg', 'min', 'max', 'volume', 'm_side']
    
    #for loop that creates new keys in the empty dictionary with a empty list as values.
    for i in nombres:
        results[i] = []
    
    #for that loops on each day in the list of the days of the month.
    for e in days:
        #variable that get's just the rows that match the day in the for loop.
        ndf = f_df.loc[f_df.day == e]
        #appending the results in each dictionary key.
        results['date'].append(datetime.date(y, m, e))
        results['24h_avg'].append(ndf.price.mean())
        results['12h_avg'].append(ndf.price.loc[ndf.hour<12].mean())
        results['max'].append(ndf.price.max())
        results['min'].append(ndf.price.min())
        results['volume'].append(ndf.volume.sum())
        weight = ndf.side.value_counts().sum()
        buy = ndf.side.value_counts()['buy']/weight
        sell = ndf.side.value_counts()['sell']/weight
        if buy>sell:
            results['m_side'].append('buy')
        else:
            results['m_side'].append('sell')
            
    #returns the dictionary with the appended elements
    return results

In [37]:
month = datetime.datetime.today().month
df_new = pd.DataFrame.from_dict(extraction(df, month))

btc_df.date = btc_df.date.astype('datetime64')
btc_df['min'] = btc_df['min'].astype('float')
btc_df['max'] = btc_df['max'].astype('float')
btc_df.volume = btc_df.volume.astype('float')
btc_df['m_side'] = btc_df['m_side'].astype('category')

df_dum = pd.get_dummies(btc_df)
df_dum.rename(columns={'m_side_buy':'buy', 'm_side_sell':'sell'}, inplace=True)
df_dum.tail()

,date,24h_avg,12h_avg,min,max,volume,buy,sell
359,2019-12-25,137141.208067,137650.575500,136107.32,138275.79,10.893816,1,0
360,2019-12-26,138124.781786,136821.435750,136177.03,140958.06,69.727543,1,0
361,2019-12-27,136204.858252,136431.786897,134136.13,137350.00,46.595173,1,0
362,2019-12-28,137952.567017,137676.135800,137201.00,138790.15,35.937653,1,0
363,2020-01-01,137245.246399,137126.333529,136480.14,137600.00,6.868854,1,0


In [7]:
#Reading our csv file where we store the past data
history = pd.read_csv('../csv_files/historic_data.csv')

In [8]:
btc_df = pd.concat([history, df_new], ignore_index=True)

In [9]:
btc_df.drop_duplicates(inplace=True)

In [10]:
btc_df.drop(btc_df.tail(1).index,inplace=True)

In [11]:
btc_df.to_csv('../csv_files/historic_data.csv', index=False)

In [12]:
btc_df.date = btc_df.date.astype('datetime64')
btc_df['min'] = btc_df['min'].astype('float')
btc_df['max'] = btc_df['max'].astype('float')
btc_df.volume = btc_df.volume.astype('float')
btc_df['m_side'] = btc_df['m_side'].astype('category')

In [13]:
df_dum = pd.get_dummies(btc_df)
df_dum.rename(columns={'m_side_buy':'buy', 'm_side_sell':'sell'}, inplace=True)

In [14]:
change = []
for i in range(1,len(df_dum)):
    if df_dum.sell.iloc[i] != df_dum.sell.iloc[i-1]:
        change.append(df_dum.date.iloc[i])

In [15]:
#Selecting the columns we need for fbprophet
btc_24h_avg = btc_df[['date', '24h_avg']].copy()
#Changing the columns names for fbprophet to accept it.
btc_24h_avg.rename(columns={'date': 'ds', '24h_avg': 'y'}, inplace=True)

In [16]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [17]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [18]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    df = df.copy()
    df.dropna(inplace=True)
    
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    predicted_part = df[-prediction_size:]
    
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [19]:
train_df = btc_24h_avg.copy().set_index('ds')

In [20]:
train_df['y'], lambda_prophet = boxcox(train_df['y'])
train_df.reset_index(inplace=True)

In [39]:
m2 = Prophet(changepoints=change, seasonality_mode='multiplicative')
m2.fit(train_df)
future2 = m2.make_future_dataframe(periods=15)
forecast2 = m2.predict(future2)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [43]:
forecast2

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2019-01-01,12070.595254,10086.298295,13814.585667,12070.595254,12070.595254,0.000855,0.000855,0.000855,0.000855,0.000855,0.000855,0.0,0.0,0.0,12080.917930
1,2019-01-02,12086.197132,10325.811698,14077.022506,12086.197132,12086.197132,0.000883,0.000883,0.000883,0.000883,0.000883,0.000883,0.0,0.0,0.0,12096.868627
2,2019-01-03,12101.799010,10218.619822,13968.559170,12101.799010,12101.799010,-0.004868,-0.004868,-0.004868,-0.004868,-0.004868,-0.004868,0.0,0.0,0.0,12042.883231
3,2019-01-04,12117.400888,10328.647103,13930.915466,12117.400888,12117.400888,-0.003841,-0.003841,-0.003841,-0.003841,-0.003841,-0.003841,0.0,0.0,0.0,12070.854290
4,2019-01-05,12133.002766,10327.411256,13963.964846,12133.002766,12133.002766,0.003557,0.003557,0.003557,0.003557,0.003557,0.003557,0.0,0.0,0.0,12176.163305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,2020-01-12,20305.418856,18543.293515,22243.168962,20244.642969,20364.639095,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.0,0.0,0.0,20306.283829
375,2020-01-13,20236.198775,18286.958891,22146.278535,20168.518733,20303.935634,0.003372,0.003372,0.003372,0.003372,0.003372,0.003372,0.0,0.0,0.0,20304.427707
376,2020-01-14,20166.978695,18360.991221,22058.806998,20090.166111,20243.497714,0.000855,0.000855,0.000855,0.000855,0.000855,0.000855,0.0,0.0,0.0,20184.225331
377,2020-01-15,20097.758614,18332.402966,22058.929757,20010.235044,20186.276187,0.000883,0.000883,0.000883,0.000883,0.000883,0.000883,0.0,0.0,0.0,20115.503908


In [22]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast2[column] = inverse_boxcox(forecast2[column],
                                       lambda_prophet)

In [23]:
cmp_df2 = make_comparison_dataframe(btc_24h_avg, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, 15).items():
     print(err_name, err_value)

MAPE 2.4353291528879817
MAE 3329.293646615347


In [24]:
def show_forecast(cmp_df, num_predictions, num_values, title):
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Zona de compra', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="green"))
    upper_bound = create_go('Máximo esperado', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="red"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Venta sugerida', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Actual', 'y', num_values,
                       marker=dict(color="red"))
    

    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='MXN'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename='btc_mxn.html', auto_open=False)

show_forecast(cmp_df2, 15, 500, 'BTC')